In [ ]:
! pip install gdown
! pip install zipfile
! pip install neurora
import numpy as np
import sys
import os
from six.moves import urllib
import gdown
import zipfile
from neurora.decoding import tbyt_decoding_kfold
from neurora.rsa_plot import plot_tbyt_decoding_acc, plot_rdm, plot_tbytsim_withstats
from neurora.rdm_cal import eegRDM
from neurora.corr_cal_by_rdm import rdms_corr

In [ ]:
data_dir = 'workshop_data/'
os.makedirs(data_dir)

print('================ Demo Data 1 ================')

id = '13bCcdHGVfU6-ahgu95d-W5hkwPAmvNB-'
filename = 'data1.zip'
filepath = data_dir + filename

# Download the data
gdown.download(id=id, output=filepath, quiet=False, fuzzy=True)
print('Download completes!')
# unzip the data
with zipfile.ZipFile(filepath, 'r') as zip:
    zip.extractall(data_dir)
print("Unzip completes!")

print('================ Demo Data 2 ================')

id = '1SaDPoY65XdgrymEznQlvVLb3M4z59wAS'
filename = 'data2.zip'
filepath = data_dir + filename

# Download the data
gdown.download(id=id, output=filepath, quiet=False, fuzzy=True)
print('Download completes!')
# unzip the data
with zipfile.ZipFile(filepath, 'r') as zip:
    zip.extractall(data_dir)
print("Unzip completes!")

data1 = np.load(data_dir + 'data1/data.npy')
labels1 = np.load(data_dir + 'data1/labels.npy')
data2 = np.load(data_dir + 'data2/data.npy')
labels2 = np.load(data_dir + 'data2/labels.npy')

### Example 1 Classification-based decoding - decode image representations (based on demo data1)

Check the data

In [ ]:
print(np.shape(data1))
print(np.shape(labels1))

make some notes here:  
10 - number of subjects  
160 - number of trials  
17 - number of channels  
100 - number of timepoints (-0.2s to 0.8s, sample frequency = 100)

In [ ]:
print(data1)

In [ ]:
print(labels1)

0 - basketball  
1 - cat

#### Decode brain representations (classification-based decoding)

Goal:
input of brain signals: 10 * 160 * 17 * 100  
output of decoding accuracy: 10 * 100  

How to do:  
for each subject  
for each timepoint  
two-class classification using linear classifier  
cross-validation

In [ ]:
# This is the easiest way to do decoding
# A ready-made function: tbyt_decoding_kfold()
help(tbyt_decoding_kfold)

In [ ]:
accuracies = tbyt_decoding_kfold(data1, labels1, n=2, navg=5, time_win=1, time_step=1, nfolds=3, nrepeats=3)

Check decoding results

In [ ]:
print(accuracies.shape)

In [ ]:
print(accuracies)

#### Plot decoding results

In [ ]:
# This is the easiest way to plot decoding results
# A ready-made function: plot_tbyt_decoding_acc()
help(plot_tbyt_decoding_acc)

In [ ]:
plot_tbyt_decoding_acc(accuracies, start_time=-0.2, end_time=0.8, time_interval=0.01, chance=0.5, p=0.05, 
                       cbpt=False, stats_time=[0, 0.8], color='r', xlim=[-0.2, 0.8], ylim=[0.4, 0.8])

In [ ]:
plot_tbyt_decoding_acc(accuracies, start_time=-0.2, end_time=0.8, time_interval=0.01, chance=0.5, p=0.05, 
                       cbpt=False, stats_time=[0, 0.8], color='r', xlim=[-0.2, 0.8], ylim=[0.4, 1])

In [ ]:
plot_tbyt_decoding_acc(accuracies, start_time=-0.2, end_time=0.8, time_interval=0.01, chance=0.5, p=0.05, 
                       cbpt=True, stats_time=[0, 0.8], color='r', xlim=[-0.2, 0.8], ylim=[0.4, 1])

### Example 2 Classsification-based decoding - decoding orientation representations (based on demo data2)

Check the data

In [ ]:
print(np.shape(data2))
print(np.shape(labels2))

make some notes here:  
5 - number of subjects  
640 - number of trials  
27 - number of channels  
500 - number of timepoints (-0.5s to 1.5s, sample frequency = 250)

In [ ]:
print(data2)

In [ ]:
print(labels2)

0-15 correspond to 16 orientations:  
0 -> 0°;  
1 -> 22.5°;  
2 -> 45°;  
3 -> 67.5°;  
...  
14 -> 315°;  
15 -> 337.5°

#### Decode brain representations (classification-based decoding)

Goal:  
input of brain signals: 5 * 640 * 27 * 500  
output of decoding accuracy: 5 * 500  

Do a modification here to save time:  
downsample the data - if we average each 5 timepoint  
sample frequency: 250Hz -> 50Hz  
input: 5 * 640 * 27 * 500  
output: 5 * 100

How to do:  
for each subject  
for each timepoint  
sixteen-class classification using linear classifier  
cross-validation

In [ ]:
accuracies = tbyt_decoding_kfold(data2, labels2, n=16, navg=5, time_win=5, time_step=5, nfolds=3, nrepeats=3)

Check decoding results

In [ ]:
print(accuracies.shape)

In [ ]:
print(accuracies)

#### Plot the decoding results

In [ ]:
plot_tbyt_decoding_acc(accuracies, start_time=-0.5, end_time=1.5, time_interval=0.02, chance=0.0625, p=0.05, 
                       cbpt=False, stats_time=[0, 1.5], color='r', xlim=[-0.2, 1.5], ylim=[0, 0.2])

In [ ]:
plot_tbyt_decoding_acc(accuracies, start_time=-0.5, end_time=1.5, time_interval=0.02, chance=0.0625, p=0.05, 
                       cbpt=False, stats_time=[0, 1.5], color='r', xlim=[-0.2, 1.5], ylim=[0.025, 0.125])

In [ ]:
plot_tbyt_decoding_acc(accuracies, start_time=-0.5, end_time=1.5, time_interval=0.02, chance=0.0625, p=0.05, 
                       cbpt=True, stats_time=[0, 1.5], color='r', xlim=[-0.2, 1.5], ylim=[0.025, 0.125])

### Example 3 RSA - decoding orientation representations (based on demo data2)

How to do  
Make hypothesis-based orientation RDM  
Make EEG RDMs for each timepoint for each subject  
Compare the orientation RDM and EEG RDMs  

5 - number of subjects  
640 - number of trials  
27 - number of channels  
500 - number of timepoints (-0.5s to 1.5s, sample frequency = 250)

#### Make orientation RDM

Assumption:  
The closer the two orientations are, the higher the similarity is (the lower the dissimilarity is).  
The less close the two orientations are, the lower the similarity is (the higher the dissimilarity is).

In [ ]:
oriRDM = np.zeros([16, 16])
for i in range(16):
    for j in range(16):
        diff = np.abs(i - j)
        if diff <= 8:
            oriRDM[i, j] = diff / 8
        else:
            oriRDM[i, j] = (16 - diff) / 8
print(oriRDM)

#### Plot the RDM

In [ ]:
conditions = ["0°", "22.5°", "45°", "67.5°", "90°", "112.5°", "135°", "157.5°", "180°",
              "202.5°", "225°", "247.5°", "270°", "292.5°", "315°", "337.5°"]

# A ready-made function: plot_rdm
plot_rdm(oriRDM, conditions=conditions)

#### Make EEG RDMs

Goal:  
input of brain signals: 5 * 640 * 27 * 500 -> 16 * 5 * 40 * 27 * 500  
output of EEG RDMs: 5 * 500 * 16 * 16  

In [ ]:
data2_16conditions = np.zeros([16, 5, 40, 27, 500])
for sub in range(5):
    index = np.zeros([16], dtype=int)
    for i in range(640):
        condition = int(labels2[sub, i])
        data2_16conditions[condition, sub, index[condition]] = data2[sub, i]
        index[condition] = index[condition] + 1

Check new data

In [ ]:
print(data2_16conditions.shape)

Goal:  
input of brain signals: 16 * 5 * 40 * 27 * 500  
output of EEG RDMs: 5 * 500 * 16 * 16  

We also downsampling the data (average every 5 timepoints, 500 timepoints -> 100 timepoints)  
output of EEG RDMs: 5 * 100 * 16 * 16

In [ ]:
# This is the easiest way to calculate RDMs
# A ready-made function: eegRDM()
help(eegRDM)

In [ ]:
eegRDMs = eegRDM(data2_16conditions, sub_opt=1, chl_opt=0, time_opt=1, time_win=5, time_step=5)

Check EEG RDMs

In [ ]:
print(eegRDMs.shape)

In [ ]:
print(eegRDMs)

#### Compare between Orientation RDM and EEG RDMs

Goal:  
inputs: orientation RDM 16 * 16 and EEG RDMs 5 * 100 * 16 * 16  
output of RSA results (representational similarities for each subject and each timepoint): 5 * 100

In [ ]:
# This is the easiest way to calculate the similarity
# A ready-made function: rdms_corr()
help(rdms_corr)

In [ ]:
similarities = rdms_corr(oriRDM, eegRDMs)

Check RSA results

In [ ]:
print(similarities.shape)

In [ ]:
similarities = similarities[:, :, 0]

In [ ]:
print(similarities)

#### Plot RSA results

In [ ]:
# This is the easiest way to plot RSA results
# A ready-made function: plot_tbytsim_withstats()
help(plot_tbytsim_withstats)

In [ ]:
plot_tbytsim_withstats(similarities, start_time=-0.5, end_time=1.5, time_interval=0.02, p=0.05,
                       cbpt=False, stats_time=[0, 1.5], xlim=[-0.5, 1.5], ylim=[-0.1, 0.8])

In [ ]:
plot_tbytsim_withstats(similarities, start_time=-0.5, end_time=1.5, time_interval=0.02, p=0.05,
                       cbpt=False, stats_time=[0, 1.5], xlim=[-0.5, 1.5], ylim=[-0.1, 0.3])

In [ ]:
plot_tbytsim_withstats(similarities, start_time=-0.5, end_time=1.5, time_interval=0.02, p=0.05,
                       cbpt=True, stats_time=[0, 1.5], xlim=[-0.5, 1.5], ylim=[-0.1, 0.3])